In [8]:
import os
from osgeo import gdal,gdalconst
import geopandas as gpd
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from shapely.geometry import box
warnings.filterwarnings("ignore")

def read_tif(tif_file):
    dataset = gdal.Open(tif_file)
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    im_proj = (dataset.GetProjection())
    im_Geotrans = (dataset.GetGeoTransform())
    im_data = dataset.ReadAsArray(0, 0, cols, rows)
    if im_data.ndim == 3:
        im_data = np.moveaxis(dataset.ReadAsArray(0, 0, cols, rows), 0, -1)
    return im_data, im_Geotrans, im_proj,rows, cols

def array_to_geotiff(array, output_path, geo_transform, projection, band_names=None):
    rows, cols, num_bands = array.shape
    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(output_path, cols, rows, num_bands, gdal.GDT_Float32)
    
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    
    for band_num in range(num_bands):
        band = dataset.GetRasterBand(band_num + 1)
        band.WriteArray(array[:, :, band_num])
        band.FlushCache()
        
        if band_names:
            band.SetDescription(band_names[band_num])
    dataset = None
    band = None
    return

In [7]:
points_file = '/130TB_RAID0/fujiang/SmallSat_part2/2_high_resolution_trait_maps/5_extract_training_samples/1_extracted_points.shp'
out = "/130TB_RAID0/fujiang/SmallSat_part2/2_high_resolution_trait_maps/5_extract_training_samples/0_sparse_vagetated_area/0_EMIT_polygan.shp"

points = gpd.read_file(points_file)


def point_to_square(point, size=60):
    half = size / 2
    return box(point.x - half, point.y - (size-11.5), point.x + half, point.y+11.5)

gdf_polygons = points.copy()
gdf_polygons['geometry'] = points.geometry.apply(point_to_square)
gdf_polygons.to_file(out)